In [ ]:
import numpy as np
import pandas as pd
import plotly.express as px
import json
from glob import glob
from pathlib import Path
from tqdm import tqdm

In [ ]:
base = "data/done-soon/temp/problem_output/"
all_normal_files = glob("data/done-soon/temp/problem_output/*NORMAL.json")
df = pd.DataFrame()
data = []

num_without_search_time = 0
for i, normal in tqdm(enumerate(all_normal_files)):
    stats = Path(f"{normal[:-12]}-STATS.json")
    if stats.exists():
        with open(normal, 'r') as normal_output, open(f"{normal[:-12]}-STATS.json", 'r') as stats_output:
            line = normal_output.readline()
            if line: # don't read json from empty output
                normal_time = json.loads(line).get('time')
                
                stats = json.loads(stats_output.readlines()[-1]).get('statistics')

                if normal_time and stats:
                    # if normal_time * 0.001 > 10:
                    if "search_time" not in stats.keys():
                        num_without_search_time += 1
                    else:
                        data.append({
                            'normal_time': normal_time * 0.001,
                            'stat_time': stats['search_time'],
                            'problem': normal
                        })
                        
df = pd.DataFrame(data)
df.shape

In [ ]:
log_axes = False
log_fig = False
scatter = px.scatter(
    df,
    x='normal_time',
    y='stat_time',
    labels={'x': 'Normal chuffed runtime', 'y': 'Modded chuffed runtime'},
    title="Runtime of Normal Chuffed vs. Modded Chuffed",
    log_x=log_axes,
    log_y=log_axes,
    # trendline='ols',
    # trendline_options=dict(log_x=fit_log, log_y=fit_log),
)
scatter.layout.update(showlegend=False)
scatter

In [ ]:
df[df['normal_time'] > 3600].shape

In [ ]:
base = "data/done-soon/temp/problem_output/"
all_normal_files = glob("data/done-soon/temp/problem_output/*-black-hole*-18-*NORMAL.json")
xs = []
ys = []
stat_times = []

for i, normal in enumerate(all_normal_files):
    stats = Path(f"{normal[:-12]}-STATS.json")
    if stats.exists():
        with open(normal, 'r') as normal_output, open(f"{normal[:-12]}-STATS.json", 'r') as stats_output:
            normal_time = json.loads(normal_output.readline()).get('time')
            stats_lines = stats_output.readlines()
            stats = json.loads(stats_lines[-1]).get('statistics')
            
            if normal_time and stats:
                if normal_time * 0.001 > 10:
                    xs.append(normal_time * 0.001)
                    ys.append(stats['search_time'])
                for line in stats_lines:
                    stats = json.loads(line).get('statistics')
                    if stats:
                        stat_times.append(stats['search_time'])

                    


In [ ]:
px.scatter(y=stat_times, labels={"x": "data point", "y": "search_time value"})

In [ ]:
import plotly.graph_objects as go

In [ ]:
base = "data/done-soon/benchmarks/solve/"
all_normal_files = glob(f"{base}/*NORMAL.tsv")
data = []


for i, normal in enumerate(all_normal_files):
    stats = Path(f"{normal[:-11]}-STATS.tsv")
    if stats.exists():
        with open(normal, 'r') as normal_output, open(stats, 'r') as stats_output:
            normal_time = float(normal_output.readlines()[-1].split('\t')[0])
            stat_time = float(stats_output.readlines()[-1].split('\t')[0])

            if normal_time and stats:
                data.append({
                    'normal_time': normal_time,
                    'stat_time': stat_time,
                    'problem': normal
                })
df_bench = pd.DataFrame(data)
df_bench.shape

In [ ]:
df_bench_filtered = df_bench  # [df_bench['stat_time'] < 7100]

In [ ]:
show_log = True
fit_log = False
fig = px.scatter(
    df_bench_filtered,
    x='normal_time',
    y='stat_time',
    # trendline='ols',
    log_x=show_log,
    log_y=show_log,
    trendline_options=dict(log_x=fit_log, log_y=fit_log),
    width=1000,
    height=1000
)
results = px.get_trendline_results(fig)
# fig.add_trace(px.line(x=[0, 7200], y=[0, 7200]).data[0])
# fig['data'][2]['line']['color']='rgb(255, 0, 0)'
# fig.update_yaxes(
#     range=[-300,7500]
# )
# fig.update_xaxes(
#     range=[-300,7500]
# )
fig

In [ ]:
results.px_fit_results.iloc[0].summary()